In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import random
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import os,sys
opj = os.path.join
from tqdm import tqdm
import acd
from copy import deepcopy
from model_mnist import LeNet5
from visualize import *
import dset_mnist as dset
import foolbox
sys.path.append('../trim')
from transforms_torch import transform_bandpass, tensor_t_augment, batch_fftshift2d, batch_ifftshift2d
from trim import *
from util import *
from attributions import *
from captum.attr import *
import warnings
warnings.filterwarnings("ignore")
# disentangled vae
sys.path.append('../disentangling-vae')
from collections import defaultdict
import vae_trim, vae_trim_viz
from disvae.utils.modelIO import save_model, load_model, load_metadata
from disvae.models.losses import get_loss_f

### Train model

In [2]:
args = vae_trim.parse_arguments()
args.loss = "btcvae"
args.reg_anneal = 0
# args.btcvae_B = 0
args.attr_lamb = 200
args.epochs = 50
name = args.loss + "_B_" + str(args.btcvae_B) + "_attr_" + str(args.attr_lamb)
# name = args.loss + "_attr_" + str(args.attr_lamb)
args.name = name

In [ ]:
# train and evaluate model
vae_trim.main(args)

15:18:21 INFO - main: Root directory for saving and loading experiments: results/btcvae_B_6_attr_200
15:18:21 INFO - main: Train mnist with 60000 samples
15:18:21 INFO - main: Num parameters in model: 469173
15:18:22 INFO - __init__: Training Device: cuda
15:22:29 INFO - __call__: Epoch: 1 Average loss per image: 77.45     
15:26:53 INFO - __call__: Epoch: 2 Average loss per image: 36.62     
15:31:16 INFO - __call__: Epoch: 3 Average loss per image: 28.88      
15:35:39 INFO - __call__: Epoch: 4 Average loss per image: 24.56      
15:39:53 INFO - __call__: Epoch: 5 Average loss per image: 22.13      
15:42:38 INFO - __call__: Epoch: 6 Average loss per image: 20.69        
15:45:00 INFO - __call__: Epoch: 7 Average loss per image: 19.73      
15:47:24 INFO - __call__: Epoch: 8 Average loss per image: 16.14      
15:49:46 INFO - __call__: Epoch: 9 Average loss per image: 15.77       
15:52:08 INFO - __call__: Epoch: 10 Average loss per image: 14.68        
Epoch 11:  75%|███████▌  | 706

In [ ]:
# generate visualization
args = vae_trim_viz.parse_arguments()
args.name = name
args.plots = "all"
args.n_rows = 10
args.n_cols = 10
vae_trim_viz.main(args)

### Test model

In [ ]:
args = vae_trim.parse_arguments()
args.loss = "VAE"
args.name = name
# results dir
exp_dir = os.path.join(vae_trim.RES_DIR, args.name)

# load dataloaders
train_loader, test_loader = dset.load_data(args.batch_size, args.eval_batchsize, device)
metadata = load_metadata(exp_dir)

# load model
model = load_model(exp_dir, is_gpu=not args.no_cuda)

In [ ]:
# loss
loss_f = get_loss_f(args.loss,
                    n_data=len(test_loader.dataset),
                    device=device,
                    **vars(args))

# evaluate on testset
storer = defaultdict(list)
for data, _ in tqdm(test_loader, leave=False, disable=args.no_progress_bar):
    data = data.to(device)
    recon_batch, latent_dist, latent_sample = model(data)
    _ = loss_f(data, recon_batch, latent_dist, model.training,
                    storer, latent_sample=latent_sample)    
    losses = {k: sum(v) / len(test_loader) for k, v in storer.items()}
    break
print(losses)